In [1]:
import os
import pandas as pd

In [71]:
path = 'C:\\Users\\Dudu_\\OneDrive\\Documentos\\Estudos\\1. Projetos Pessoais\\3. Previsão Público Cruzeiro\\data\\raw\\climate_data'

In [72]:
def load_excel(path):

    arquivos = os.listdir(path)

    # Lista para armazenar os dfs
    df_stacked = None
    colunas = None
    # Loop sobre os arquivos na pasta
    for n, arquivo in enumerate(arquivos):
        # Verifica se o arquivo é um arquivo CSV
        if arquivo.endswith('.CSV'):

            # Constrói o caminho completo para o arquivo
            caminho_arquivo = os.path.join(path, arquivo)
            
            # Leitura do arquivo CSV e pula as primeiras 8 linhas
            df = pd.read_csv(caminho_arquivo, encoding='latin-1', delimiter=';', header=None, skiprows=8)
            filtered_df = df[[0,1,2,7,8,18]]

            if n == 0:
                filtered_df.columns = filtered_df.iloc[0]
                colunas = filtered_df.columns
                filtered_df = filtered_df.drop(index=0)
                df_stacked = filtered_df.copy()
            else:
                filtered_df.columns = colunas
                filtered_df = filtered_df.drop(index=0)
                df_stacked = pd.concat([df_stacked, filtered_df], ignore_index=True)
    
    return df_stacked


In [73]:
load_excel(path)

,Data,Hora UTC,"PRECIPITAÇÃO TOTAL, HORÁRIO (mm)","TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)",TEMPERATURA DO PONTO DE ORVALHO (°C),"VENTO, VELOCIDADE HORARIA (m/s)"
0,2021/01/01,0000 UTC,",2","20,8","19,2","1,2"
1,2021/01/01,0100 UTC,0,"21,7",19,"2,1"
2,2021/01/01,0200 UTC,0,"21,6",19,"1,5"
3,2021/01/01,0300 UTC,0,"22,3","18,4","1,6"
4,2021/01/01,0400 UTC,0,"22,5","17,8","1,5"
...,...,...,...,...,...,...
105187,2020/12/31,1900 UTC,0,"29,6","16,2","1,2"
105188,2020/12/31,2000 UTC,0,"29,2","15,6",",6"
105189,2020/12/31,2100 UTC,0,28,"15,5","1,3"
105190,2020/12/31,2200 UTC,0,"27,6","15,2",",6"
